# Seldon deployment of income classifier and Alibi anchor explainer

The objective of this tutorial is to build a "loan approval" predictor using the Income classifier dataset to showcase the importance of black-box model explainers, which in this case are built using our open source framework [Alibi](http://github.com/SeldonIO/Alibi). The diagram of this tutorial is as follows:

![deploy-overview](https://github.com//SeldonIO/seldon-core/raw/master/examples/explainers/alibi_anchor_tabular/img/deploy-overview.jpg)

In this tutorial we will follow the following steps:

1) Train a model to predict loan approvals

2) Containerise and deploy your model

3) Create an explainer to understand predictions

4) Containerise and deploy your explainer

5) Test the predictions as well as explanations

## Before you start
Make sure you install the following dependencies, as they are critical for this example to work:

* Helm v3.0.0+
* A Kubernetes cluster running v1.13 or above (minkube / docker-for-windows work well if enough RAM)
* kubectl v1.14+
* ksonnet v0.13.1+
* kfctl 0.5.1 - Please use this exact version as there are major changes every few months
* Python 3.6+
* Python DEV requirements (we'll install them below)

You can follow this [notebook](../../../notebooks/seldon_core_setup.ipynb) to setup your cluster.

Let's get started! 🚀🔥 


### Install python dependencies

In [1]:
!cat requirements-dev.txt

xai==0.0.5
seldon_core==0.5.1
alibi==0.3.2
dill==0.3.1
scikit-learn==0.20.1


In [2]:
!pip install -r requirements-dev.txt

     |████████████████████████████████| 358kB 961kB/s eta 0:00:01
     |████████████████████████████████| 61kB 3.0MB/s eta 0:00:011
     |████████████████████████████████| 92kB 7.4MB/s  eta 0:00:01
     |████████████████████████████████| 153kB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 5.4MB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 31.2MB 6.1MB/s eta 0:00:011
     |████████████████████████████████| 512kB 4.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 2.8MB/s eta 0:00:011
     |████████████████████████████████| 92kB 4.0MB/s eta 0:00:011
     |████████████████████████████████| 8.8MB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 12.9MB 6.8MB/s eta 0:00:01
     |████████████████████████████████| 13.9MB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 235kB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 26.5MB 26.4MB/s eta 0:00:01
     |████████████████████████████████| 112kB 16.0MB/s eta 0:00:01


     |████████████████████████████████| 110.5MB 199kB/s eta 0:00:011
     |████████████████████████████████| 3.3MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 12.0MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 512kB 19.6MB/s eta 0:00:01


  Created wheel for dill: filename=dill-0.3.1.dev0-cp37-none-any.whl size=78554 sha256=3f80d78a42751f123bdcd71ceab39f7de5f1b42bad0cdce3d5f9f762f8f76097
  Stored in directory: /home/alejandro/.cache/pip/wheels/b6/26/8f/152327a2b78a0c2c3166e5d20d331bd4fb1272e810836fed76
Successfully built dill
ERROR: tensorflow 1.15.2 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: creme 0.4.4 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: creme 0.4.4 has requirement scikit-learn>=0.21.2, but you'll have scikit-learn 0.20.1 which is incompatible.
ERROR: creme 0.4.4 has requirement scipy>=1.3.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: alibi-detect 0.1.0 has requirement tensorflow>=2, but you'll have tensorflow 1.15.2 which is incompatible.
  Found existing installation: numpy 1.17.4
    Uninstalling numpy-1.17.4:
      Successfully uninstalled numpy-1.17.4
  Found existing installation: scipy 1.3.3


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## 1) Train a model to predict loan approvals 

In [3]:
import alibi
import numpy as np

adult = alibi.datasets.fetch_adult()

data = adult.data
labels = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

# define train and test set
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, labels])
data = data_perm[:, :-1]
labels = data_perm[:, -1]

idx = 30000
X_train, y_train = data[:idx, :], labels[:idx]
X_test, y_test = data[idx + 1:, :], labels[idx + 1:]

In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# feature transformation pipeline
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                      ('scaler', StandardScaler())])

categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer, ordinal_features),
                                               ('cat', categorical_transformer, categorical_features)])

In [5]:
preprocessor.fit(data)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]), [0, 8, 9, 10]), ('cat', Pipeline(memory=None,
     steps=[(...oat64'>, handle_unknown='ignore',
       n_values=None, sparse=True))]), [1, 2, 3, 4, 5, 6, 7, 11])])

In [6]:
from sklearn.ensemble import RandomForestClassifier

np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(preprocessor.transform(X_train), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
import xai
pred = clf.predict(preprocessor.transform(X_test))
xai.metrics_plot(y_test, pred)

,target
precision,0.708042
recall,0.661765
specificity,0.914271
accuracy,0.853906
auc,0.788018
f1,0.684122


## 2) Containerise and deploy your model

The steps to cotainerise a model with Seldon are always consistent, and require the following steps:

1) Save the model artefacts in the model folder

2) Write a wrapper with a `predict` function

3) Add the python requirements

4) Add the Source2Image configuration so the script knows which Model.py file to use

5) Run the s2i command to build the image

6) Deploy your image with a Seldon Graph Definition

### Once you've deployed it, you are able to test it with Curl or with our Python SeldonClient

Let's start containerising it - we'll be using the following folder for this:

In [1]:
!mkdir -p pipeline/pipeline_steps/loanclassifier

### 2.1 - Save the trained model in the folder 

In [2]:
import dill

with open("pipeline/pipeline_steps/loanclassifier/preprocessor.dill", "wb") as prep_f:
    dill.dump(preprocessor, prep_f)
    
with open("pipeline/pipeline_steps/loanclassifier/model.dill", "wb") as model_f:
    dill.dump(clf, model_f)

NameError: name 'preprocessor' is not defined

### 2.2 - Write a python wrapper for the loan approval model

In [3]:
%%writefile pipeline/pipeline_steps/loanclassifier/Model.py
import dill
import logging

class Model:
    def __init__(self, *args, **kwargs):
        
        with open("preprocessor.dill", "rb") as prep_f:
            self.preprocessor = dill.load(prep_f)
        with open("model.dill", "rb") as model_f:
            self.clf = dill.load(model_f)
        
    def predict(self, X, feature_names=[]):
        logging.warn("Received: " + str(X))
        X_prep = self.preprocessor.transform(X)
        proba = self.clf.predict_proba(X_prep)
        logging.warn("Predicted: " + str(proba))
        return proba

Writing pipeline/pipeline_steps/loanclassifier/Model.py


### 2.3 - Add the python requirements for the image

In [4]:
%%writefile pipeline/pipeline_steps/loanclassifier/requirements.txt
scikit-learn==0.20.1
dill==0.3.1
scikit-image==0.15.0
scikit-learn==0.20.1
scipy==1.1.0
numpy==1.15.4

Writing pipeline/pipeline_steps/loanclassifier/requirements.txt


### 2.4 - Create the source2image configuration file

In [5]:
!mkdir -p pipeline/pipeline_steps/loanclassifier/.s2i

In [6]:
%%writefile pipeline/pipeline_steps/loanclassifier/.s2i/environment
MODEL_NAME=Model
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0

Writing pipeline/pipeline_steps/loanclassifier/.s2i/environment


### 2.5 - Now we can build the image

In [32]:
!s2i build pipeline/pipeline_steps/loanclassifier seldonio/seldon-core-s2i-python37:0.18 loanclassifier:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Created wheel for dill: filename=dill-0.3.1.dev0-cp37-none-any.whl size=78554 sha256=aeb09938c18110175f46b023dbcf15f93f98bb79bc0c5eeda7804da725c8f327
Stored in directory: /root/.cache/pip/wheels/b6/26/8f/152327a2b78a0c2c3166e5d20d331bd4fb1272e810836fed76
Successfully built dill
Found existing installation: numpy 1.18.1
Uninstalling numpy-1.18.1:
Successfully uninstalled numpy-1.18.1
Build completed successfully


*or* if using Minikube

### 2.6 - And deploy it to Kubernetes

In [7]:
%%writefile pipeline/pipeline_steps/loanclassifier/loanclassifiermodel.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: loanclassifier
spec:
  name: loanclassifier
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: loanclassifier:0.1
          name: model
    graph:
      children: []
      name: model
      type: MODEL
      endpoint:
        type: REST
    name: loanclassifier
    replicas: 1

Writing pipeline/pipeline_steps/loanclassifier/loanclassifiermodel.yaml


In [20]:
!kubectl apply -f pipeline/pipeline_steps/loanclassifier/loanclassifiermodel.yaml

seldondeployment.machinelearning.seldon.io/loanclassifier created


In [21]:
!kubectl get pods

NAME                                                              READY   STATUS    RESTARTS   AGE
default-broker-filter-7ffddb5dcc-pdbdt                            1/1     Running   0          6d
default-broker-ingress-5cfc4c8cbc-dqr82                           1/1     Running   0          6d
loanclassifier-loanclassifier-0-65f87677f6-bgt8x                  0/2     Running   0          2s
pingsource-test-ping-sourc-472a7c94-49b9-4996-83d4-1b60c0474tpm   1/1     Running   0          6d
seldon-controller-manager-5cff67b4f5-frvq6                        1/1     Running   0          3d15h
sklearn-default-0-b79d4d97d-7hvln                                 2/2     Running   0          3d15h


### Now that it's deployed we can test it with curl
**IMPORTANT:** If you are using minikube (instead of docker desktop) you have to forward the port first with:
```
kubectl port-forward svc/ambassador 8003:80
```

In [22]:
# We'll use the output of the first item:
X_test[:1]

array([[52,  4,  0,  2,  8,  4,  2,  0,  0,  0, 60,  9]])

In [35]:
%%bash
curl -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"names": ["text"], "ndarray": [[52,  4,  0,  2,  8,  4,  2,  0,  0,  0, 60, 9]]}}' \
    http://localhost:80/seldon/default/loanclassifier/api/v1.0/predictions

{"data":{"names":["t:0","t:1"],"ndarray":[[0.86,0.14]]},"meta":{}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   158  100    67  100    91   2913   3956 --:--:-- --:--:-- --:--:--  6869


### And we can also test it with the Python SeldonClient

In [39]:
from seldon_core.seldon_client import SeldonClient

batch = X_test[:1]

sc = SeldonClient(
    gateway="ambassador", 
    gateway_endpoint="localhost:80",
    deployment_name="loanclassifier",
    payload_type="ndarray",
    namespace="default",
    transport="rest")

client_prediction = sc.predict(data=batch)

print(client_prediction)

Success:True message:
Request:
data {
  ndarray {
    values {
      list_value {
        values {
          number_value: 52.0
        }
        values {
          number_value: 4.0
        }
        values {
          number_value: 0.0
        }
        values {
          number_value: 2.0
        }
        values {
          number_value: 8.0
        }
        values {
          number_value: 4.0
        }
        values {
          number_value: 2.0
        }
        values {
          number_value: 0.0
        }
        values {
          number_value: 0.0
        }
        values {
          number_value: 0.0
        }
        values {
          number_value: 60.0
        }
        values {
          number_value: 9.0
        }
      }
    }
  }
}

Response:
meta {
}
data {
  names: "t:0"
  names: "t:1"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.86
        }
        values {
          number_value: 0.14
        }
      }
    }
  }
}



## 3) Create an explainer to understand predictions

In [40]:
from alibi.explainers import AnchorTabular

predict_fn = lambda x: clf.predict(preprocessor.transform(x))
explainer = AnchorTabular(predict_fn, feature_names, categorical_names=category_map)

In [41]:
explainer.fit(X_train, disc_perc=[25, 50, 75])

In [42]:
idx = 0
class_names = ['<=50K', '>50K']
predict_fn(X_test[idx].reshape(1, -1))

array([0])

In [43]:
X_train[:1]

array([[27,  4,  4,  2,  1,  4,  4,  0,  0,  0, 44,  9]])

In [44]:
explanation = explainer.explain(X_test[idx], threshold=0.95)

print('Anchor: %s' % (' AND '.join(explanation['names'])))
print('Precision: %.2f' % explanation['precision'])
print('Coverage: %.2f' % explanation['coverage'])

Anchor: Marital Status = Separated AND Sex = Female
Precision: 0.96
Coverage: 0.11


### However we need to explain our remotely deployed model in production

For this we can actually create a `predict_remote_fn` that uses our SeldonClient to interact with the production model

In [45]:
from seldon_core.utils import get_data_from_proto

def predict_remote_fn(X):
    from seldon_core.seldon_client import SeldonClient
    from seldon_core.utils import get_data_from_proto
    
    kwargs = { 
        "deployment_name": "loanclassifier",
        "payload_type": "ndarray",
        "namespace": "default",
        "transport": "rest"
    }
    
    try:
        kwargs["gateway_endpoint"] = "localhost:80"
        sc = SeldonClient(**kwargs, gateway="ambassador")
        prediction = sc.predict(data=X)
    except:
        # If we are inside the container, we need to reach the ambassador service directly
        kwargs["gateway_endpoint"] = "localhost:8000"
        sc = SeldonClient(**kwargs, gateway="seldon")
        prediction = sc.predict(data=X)
    
    y = get_data_from_proto(prediction.response)
    return y

# So the anchor is now connected with the remote model
explainer = AnchorTabular(predict_remote_fn, feature_names, categorical_names=category_map)

#### We train the anchor explainer with the remote model

In [46]:
explainer.fit(X_train, disc_perc=[25, 50, 75])

#### We now can get explanations of the remote model

In [47]:
explanation = explainer.explain(X_test[idx], threshold=0.95)

print('Anchor: %s' % (' AND '.join(explanation['names'])))
print('Precision: %.2f' % explanation['precision'])
print('Coverage: %.2f' % explanation['coverage'])

Anchor: Marital Status = Separated AND Sex = Female
Precision: 0.96
Coverage: 0.11


## 4) Containerise and deploy your explainer

Once again we will follow the same steps to cotainerise a model with Seldon are always consistent, and require the following steps:

1) Save the model artefacts in the model folder

2) Write a wrapper with a `predict` function

3) Add the python requirements

4) Add the Source2Image configuration so the script knows which Model.py file to use

5) Run the s2i command to build the image

6) Deploy your image with a Seldon Graph Definition

### Once you've deployed it, you are able to test it with Curl or with our Python SeldonClient

Let's start containerising it - we'll be using the following folder for this:

In [9]:
!mkdir -p pipeline/pipeline_steps/loanclassifier-explainer

#### 1) Save the model artefacts in the model folder

In [49]:
import dill

with open("pipeline/pipeline_steps/loanclassifier-explainer/explainer.dill", "wb") as x_f:
    dill.dump(explainer, x_f)

#### 2) Write a wrapper with a `predict` function

In [10]:
%%writefile pipeline/pipeline_steps/loanclassifier-explainer/Explainer.py
import dill
import json
import numpy as np

class Explainer:
    def __init__(self, *args, **kwargs):
        
        with open("explainer.dill", "rb") as x_f:
            self.explainer = dill.load(x_f)
        
    def predict(self, X, feature_names=[]):
        print("Received: " + str(X))
        explanation = self.explainer.explain(X)
        print("Predicted: " + str(explanation))
        return json.dumps(explanation, cls=NumpyEncoder)

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (
        np.int_, np.intc, np.intp, np.int8, np.int16, np.int32, np.int64, np.uint8, np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

Writing pipeline/pipeline_steps/loanclassifier-explainer/Explainer.py


#### 3) Add the python requirements

In [11]:
%%writefile pipeline/pipeline_steps/loanclassifier-explainer/requirements.txt
scikit-learn==0.20.1
dill==0.3.1.
alibi==0.3.2

Writing pipeline/pipeline_steps/loanclassifier-explainer/requirements.txt


#### 4) Add the Source2Image configuration so the script knows which Model.py file to use

In [12]:
!mkdir pipeline/pipeline_steps/loanclassifier-explainer/.s2i

In [13]:
%%writefile pipeline/pipeline_steps/loanclassifier-explainer/.s2i/environment
MODEL_NAME=Explainer
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0

Writing pipeline/pipeline_steps/loanclassifier-explainer/.s2i/environment


#### 5) Run the s2i command to build the image

In [76]:
!s2i build pipeline/pipeline_steps/loanclassifier-explainer seldonio/seldon-core-s2i-python37:0.18 loanclassifier-explainer:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl


Created wheel for dill: filename=dill-0.3.1.dev0-cp37-none-any.whl size=78554 sha256=242d5143f997752b491e0f890bd55bd60ad5351ec94651b63542ac115293d147
Stored in directory: /root/.cache/pip/wheels/b6/26/8f/152327a2b78a0c2c3166e5d20d331bd4fb1272e810836fed76
Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4832 sha256=feb5d2759c779284519fa456c0c7a01dfae80a99c6b7cef7e6096ec5c924a04d
Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Created wheel for absl-py: filename=absl_py-0.9.0-cp37-none-any.whl size=121932 sha256=cc81dcdd0dec7e6800b94712207cb6797d65b397664cc2503d0fe5f0fa7f85eb
Stored in directory: /root/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=94683e29c7bba475e81aac5c7253e062994dc326bfc2daa784a77220e06174c5
Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced

*or* if using minikube

In [55]:
%%bash
eval $(minikube docker-env)
s2i build pipeline/pipeline_steps/loanclassifier-explainer seldonio/seldon-core-s2i-python37:0.18 loanclassifier-explainer:0.1

Process is terminated.


#### 6) Deploy your image with a Seldon Graph Definition

In [14]:
%%writefile pipeline/pipeline_steps/loanclassifier/loanclassifiermodel.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: loanclassifier
spec:
  name: loanclassifier
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: loanclassifier:0.1
          name: model
    graph:
      children: []
      name: model
      type: MODEL
      endpoint:
        type: REST
    explainer:
      type: "AnchorExplainer"
      containerSpec:
        name: "loanclassifier-explainer"
        image: loanclassifier-explainer:0.1
    name: loanclassifier
    replicas: 1

Overwriting pipeline/pipeline_steps/loanclassifier/loanclassifiermodel.yaml


In [68]:
!kubectl apply -f pipeline/pipeline_steps/loanclassifier/loanclassifiermodel.yaml

seldondeployment.machinelearning.seldon.io/loanclassifier created


In [60]:
!kubectl get pods

NAME                                                              READY   STATUS    RESTARTS   AGE
ambassador-69b784f9d5-242jz                                       1/1     Running   0          9m7s
ambassador-69b784f9d5-b2655                                       1/1     Running   0          9m7s
ambassador-69b784f9d5-ckrt2                                       1/1     Running   0          9m7s
loanclassifier-explainer-loanclassifier-explainer-8444816-fvb8r   2/2     Running   0          33s
loanclassifier-loanclassifier-164157f-645c997d57-vqjv4            2/2     Running   0          9m3s


### Now that it's deployed we can query it
**IMPORTANT:** If you are using minikube (instead of docker desktop) you have to forward the port first with:
```
kubectl port-forward svc/ambassador 8003:80
```

#### First we can try Curl

In [66]:
%%bash
curl -v -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"names": ["text"], "ndarray": [[52,  4,  0,  2,  8,  4,  2,  0,  0,  0, 60, 9]]}}' \
    http://localhost:80/seldon/default/loanclassifier/api/v1.0/predictions

{"data":{"names":["t:0","t:1"],"ndarray":[[0.86,0.14]]},"meta":{}}


Note: Unnecessary use of -X or --request, POST is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 80 (#0)
> POST /seldon/default/loanclassifier/api/v1.0/predictions HTTP/1.1
> Host: localhost
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 91
> 
} [91 bytes data]
* upload completely sent off: 91 out of 91 bytes
< HTTP/1.1 200 OK
< content-type: application/json
< date: Mon, 16 Mar 2020 14:44:08 GMT
< content-length: 67
< x-envoy-upstream-service-time: 12
< server: istio-envoy
< 
{ [67 bytes data]
100   158  100    67  100    91   3045   4136 --:--:-- --:--:-- --:--:--  7523
* Connection #0 to host localhost left intact


## 5) Test production predictions and explanations

We create a seldon client to send requests to the deployed model as well as the explainer. Here is the diagram of the deployed models:

![](img/deploy-overview.jpg)


In [62]:
sc = SeldonClient(
    gateway="ambassador", 
    gateway_endpoint="localhost:8003",
    payload_type="ndarray",
    namespace="seldon",
    transport="rest")

### Let's have a look at the datapoint we'll use for this prediction

In [63]:
to_explain = X_test[:1]
print(to_explain)

[[52  4  0  2  8  4  2  0  0  0 60  9]]


### We get the prediction from the model in production

In [64]:
resp = sc.predict(data=to_explain, deployment_name="loanclassifier").response
pred = get_data_from_proto(resp)
print('Predicted Label: %s' % ("POSITIVE" if pred[0][0] < 0.5 else "NEGATIVE"))
print('Predicted Probabilities: %s' % pred[0])

Predicted Label: NEGATIVE
Predicted Probabilities: [0.86 0.14]


### By checking our test label, we can see it is indeed correct

In [65]:
print('Actual Label: %s' % ("POSITIVE" if y_test[0] == 1 else "NEGATIVE"))

Actual Label: NEGATIVE


### Now we can use our deployed explainer to explain our prediction

In [66]:
import json
explanation = sc.predict(data=to_explain, deployment_name="loanclassifier-explainer")
exp = json.loads(explanation.response.strData)

print('Anchor: %s' % (' AND '.join(exp['names'])))

Anchor: Marital Status = Separated AND Sex = Female
